# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

# LLM모델에 출력 형식을 설정

- ChatModel객체의 `with_structured_output(pydantic.BaseModel)` 을 이용해 모델의 출력 형식을 모델 자체에 추가할 수있다.
- `OutputParser`는 모델의 출력 결과를 받아서 형식을 변경해 준다. 그래서 Chain에 탈/부착을 통해 형식을 적용하거나 적용하지 않는 것을 자유롭게 할 수있다.
- 모델의 출력 결과를 항상 일정하게 할 경우에는 아예 **모델에 출력 형식을 설정할 수 있다.**

# Streaming 방식 응답 처리

- Streaming 방식 응답 처리란, LLM이 텍스트를 모두 생성할 때까지 기다리지 않고, 생성되는 즉시 **부분적인 결과**를 실시간으로 전달받아 처리하는 방식을 의미한다. 이는 사용자가 응답을 더 빠르게 인지할 수 있게 해 주며, 특히 대화형 서비스, 실시간 UI 출력, 긴 문서 생성과 같은 상황에서 매우 유용하게 활용된다.

- `invoke()` 요청으로 받는 응답은 **비 스트리밍 방식**으로 모든 응답 텍스트 생성이 완료된 이후 그 결과를 한 번에 반환하는 구조이다. 반면 Streaming 방식은 **토큰(token) 단위 또는 여러 토큰이 묶인 청크(chunk) 단위**로 연속적인 데이터 스트림을 전송한다는 점에서 큰 차이가 있다. 즉, Streaming은 마치 사람이 타이핑을 치듯이 응답을 실시간으로 “흘려보내는” 방식이라고 이해할 수 있다.

- `모델.invoke(input, config)` → 응답 데이터
    - 모델이 전체 응답을 모두 생성한 뒤, 최종 결과를 한 번에 반환하는 방식이다.
    - 배치 처리나 후처리가 중요한 경우에 적합하다.
- `모델.stream(input, config)` → Iterator
    - 모델이 토큰을 생성하는 즉시, 순차적으로 결과를 제공하는 Iterator 형태로 반환한다.
    - 실시간 출력, 대화형 인터페이스, 웹 스트리밍 등에 특히 적합하다.